In [1]:
import requests

import pandas as pd
from PIL import Image
import io
import numpy as np
np.random.seed(0)

plankton_names = [
        "aphanizomenon",
        "asplanchna",
        "asterionella",
        "bosmina",
        "brachionus",
        "ceratium",
        "chaoborus",
        "conochilus",
        "copepod_skins",
        "cyclops",
        "daphnia",
        "daphnia_skins",
        "diaphanosoma",
        "diatom_chain",
        "dinobryon",
        "dirt",
        "eudiaptomus",
        "filament",
        "fish",
        "fragilaria",
        "hydra",
        "kellicottia",
        "keratella_cochlearis",
        "keratella_quadrata",
        "leptodora",
        "maybe_cyano",
        "nauplius",
        "paradileptus",
        "polyarthra",
        "rotifers",
        "synchaeta",
        "trichocerca",
        "unknown",
        "unknown_plankton",
        "uroglena",
    ]
def retrieve_images(plankton):
    repo_url_base = "https://github.com/juleshenry/quantum-mnist/blob/main/data/zooplankton_0p5x/"
    photo_prefix = "SPC-EAWAG-0P5X-"
    file_suffix = ".jpeg"
    raw_suffix = "?raw=True"
    imgs = []
    repo_data = repo_url_base + plankton + "/training_data/"
    print("Retrieving..." + repo_data)
    r = requests.get(repo_data)
    for o in str(r.content).split(photo_prefix)[1:49]:
        img_url = (
            repo_data
            + photo_prefix
            + o.split(file_suffix)[0]
            + file_suffix
            + raw_suffix
        )
        # print("Image@", img_url)
        img = requests.get(img_url)
        x = Image.open(io.BytesIO(img.content))
        imgs.append(x)
    return imgs

In [2]:
# Simple helper to assign data
def split_np_array_train_test(arr, train_test_ratio:float = 0.75):
    split = int(round(len(arr)*train_test_ratio,0))
    np.random.shuffle(arr)
    return (arr[:split],arr[split:],)

In [3]:
# Simple helper to resize images bilinearly
import math
def bilinear_interpolation(image, y, x):
    height = image.shape[0]
    width = image.shape[1]

    x1 = max(min(math.floor(x), width - 1), 0)
    y1 = max(min(math.floor(y), height - 1), 0)
    x2 = max(min(math.ceil(x), width - 1), 0)
    y2 = max(min(math.ceil(y), height - 1), 0)

    a = float(image[y1, x1])
    b = float(image[y2, x1])
    c = float(image[y1, x2])
    d = float(image[y2, x2])

    dx = x - x1
    dy = y - y1

    new_pixel = a * (1 - dx) * (1 - dy)
    new_pixel += b * dy * (1 - dx)
    new_pixel += c * dx * (1 - dy)
    new_pixel += d * dx * dy
    return round(new_pixel)


def bl_resize(image, new_height, new_width):
    new_image = np.zeros((new_height, new_width), image.dtype)  # new_image = [[0 for _ in range(new_width)] for _ in range(new_height)]

    orig_height = image.shape[0]
    orig_width = image.shape[1]

    # Compute center column and center row
    x_orig_center = (orig_width-1) / 2
    y_orig_center = (orig_height-1) / 2

    # Compute center of resized image
    x_scaled_center = (new_width-1) / 2
    y_scaled_center = (new_height-1) / 2

    # Compute the scale in both axes
    scale_x = orig_width / new_width;
    scale_y = orig_height / new_height;

    for y in range(new_height):
        for x in range(new_width):
            x_ = (x - x_scaled_center) * scale_x + x_orig_center
            y_ = (y - y_scaled_center) * scale_y + y_orig_center
            new_image[y, x] = bilinear_interpolation(image, y_, x_)
    return new_image

In [13]:
from IPython import display
QUBIT_DIMS = (16,16,)

p0 = plankton_names[0]
def preprocess_images(image):
    x = image.convert("L") #grayscale
    x = np.asarray(x) # numpy array
    x = bl_resize(x, *QUBIT_DIMS) #resize
    x = x / 255.0 #normalize
    return x
ims = [retrieve_images(pn) for pn in plankton_names]
ims = [[preprocess_images(ix) for ix in imlist] for imlist in ims] 
cartesian_ims = [(a,b,) for ai,a in enumerate(ims) for bi,b in enumerate(ims) if ai!=bi]

In [7]:
import tensorflow as tf
def create_fair_classical_model():
    # A simple model based off LeNet from https://keras.io/examples/mnist_cnn/
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Flatten(input_shape=(*QUBIT_DIMS,1)))
    model.add(tf.keras.layers.Dense(2, activation='relu'))
    model.add(tf.keras.layers.Dense(1))
    return model

In [18]:
BATCH_SIZE = 32
def make_test_train(ims: list, p: str, split_ratio:float=.75):
  split = int(len(ims)*split_ratio)
  x_train, x_test = ims[:split], ims[split:] 
  y_train, y_test = np.array([p]*len(x_train)), np.array([p]*len(x_test))
  return x_train,x_test,y_train,y_test
def shuffle(x,y):
  shuffle_ix = np.random.permutation(len(x))
  x = np.array([x[i] for i in shuffle_ix])
  y = np.array([y[i] for i in shuffle_ix])
  return x, y
pn = plankton_names
cartesian_names = [(a,b,) for a in pn for b in pn if a!=b]
for ims_names, ims_ix in zip(cartesian_names,cartesian_ims):
  print(ims_names[0],'vs.',ims_names[1])
  ims_i,ims_j = ims_ix
  data_i = make_test_train(ims_i,0)
  data_j = make_test_train(ims_j,1)
  x_train = np.array(data_i[0] + data_j[0])
  x_test = np.array(data_i[1] + data_j[1])
  y_train = np.append(data_i[2] , data_j[2])
  y_test = np.append(data_i[3] , data_j[3])
  x_train, y_train = shuffle(x_train, y_train)
  x_test, y_test = shuffle(x_test, y_test)

  model = create_fair_classical_model()
  model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                optimizer=tf.keras.optimizers.Adam(),
                metrics=['accuracy'])
  model.fit(x_train,
          y_train,
          batch_size=BATCH_SIZE,
          epochs=20,
          verbose=0,
          validation_data=(x_test, y_test),
          )

  fair_nn_results = model.evaluate(x_test, y_test,return_dict=True)
  print(fair_nn_results['loss'],fair_nn_results['accuracy'])  

aphanizomenon vs. asplanchna
1/1 [==============================] - 0s 19ms/step - loss: 0.6821 - accuracy: 0.5000
0.682060718536377 0.5
aphanizomenon vs. asterionella
1/1 [==============================] - 0s 21ms/step - loss: 0.6808 - accuracy: 0.5000
0.6808075308799744 0.5
aphanizomenon vs. bosmina
1/1 [==============================] - 0s 19ms/step - loss: 0.7082 - accuracy: 0.6250
0.708188533782959 0.625
aphanizomenon vs. brachionus
1/1 [==============================] - 0s 23ms/step - loss: 0.5986 - accuracy: 0.7500
0.5985684990882874 0.75
aphanizomenon vs. ceratium
1/1 [==============================] - 0s 23ms/step - loss: 0.7082 - accuracy: 0.5000
0.7081944346427917 0.5
aphanizomenon vs. chaoborus
1/1 [==============================] - 0s 21ms/step - loss: 0.5912 - accuracy: 0.6000
0.5912244319915771 0.6000000238418579
aphanizomenon vs. conochilus
1/1 [==============================] - 0s 20ms/step - loss: 0.6924 - accuracy: 0.3750
0.6923899054527283 0.375
aphanizomenon vs. co

KeyboardInterrupt: ignored